In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics.scorer import make_scorer
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
import pickle
from sklearn.externals import joblib 
import xgboost as xgb

print('Readyyy')

Readyyy


In [25]:
train_df.columns

Index(['Total_Stops', 'Price', 'Date', 'Month', 'M', 'Sa', 'Su', 'Th', 'Tu',
       'W', 'Dep_hrs', 'Dep_min', 'No info', 'other', 'Duration_min',
       'src__Chennai', 'src__Delhi', 'src__Kolkata', 'src__Mumbai',
       'dest__Cochin', 'dest__Delhi', 'dest__Hyderabad', 'dest__Kolkata',
       'Air India', 'IndiGo', 'Jet Airways', 'Multiple carriers', 'SpiceJet',
       'Vistara', 'other_flight'],
      dtype='object')

In [24]:
train_df = pd.read_excel('train_df.xlsx')
test_df = pd.read_excel('test_df.xlsx')

X = train_df.drop(columns = ['Price'])
y = train_df.Price

In [3]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 7)

## XGBRegressor

In [4]:
xgb_reg = xgb.XGBRegressor(n_estimators = 1000, 
                           random_state = 7, 
                           max_depth = 4, 
                           learning_rate = 0.04, 
                           objective = 'reg:squarederror',
                           booster = 'gbtree',
                           )

In [5]:
xgb_reg.fit(X_train, y_train)

/home/boi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0,
             importance_type='gain', learning_rate=0.04, max_delta_step=0,
             max_depth=4, min_child_weight=1, missing=None, n_estimators=1000,
             n_jobs=1, nthread=None, objective='reg:squarederror',
             random_state=7, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
             seed=None, silent=None, subsample=1, verbosity=1)

In [6]:
xgb_reg.score(X_train, y_train)

0.9240072832460248

In [7]:
xgb_reg.score(X_val, y_val)

0.8807096396569429

In [8]:
from sklearn.ensemble import VotingRegressor

In [9]:
rfr_reg = joblib.load('rf_regressor.pkl')

In [10]:
rfr_reg.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=3, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=1000,
                      n_jobs=None, oob_score=False, random_state=7, verbose=0,
                      warm_start=False)

In [11]:
rfr_reg.score(X_val, y_val)

0.8830244619113752

In [12]:
voting = VotingRegressor(estimators = [('rfr', rfr_reg), ('xgb', xgb_reg)])

In [13]:
voting.fit(X_train, y_train)

/home/boi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


VotingRegressor(estimators=[('rfr',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=3,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=1000,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=7, verbose=0,
           

In [14]:
voting.score(X_train, y_train), voting.score(X_val, y_val)

(0.9353096300674174, 0.8930409558406979)

In [15]:
voting.fit(X, y)

/home/boi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \
/home/boi/anaconda3/lib/python3.7/site-packages/xgboost/core.py:588: FutureWarning: Series.base is deprecated and will be removed in a future version
  data.base is not None and isinstance(data, np.ndarray) \


VotingRegressor(estimators=[('rfr',
                             RandomForestRegressor(bootstrap=True,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=3,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=1000,
                                                   n_jobs=None, oob_score=False,
                                                   random_state=7, verbose=0,
           

In [16]:
X_test = test_df.drop(columns = ['Price'])
X_test.head()

,Total_Stops,Date,Month,M,Sa,Su,Th,Tu,W,Dep_hrs,...,dest__Delhi,dest__Hyderabad,dest__Kolkata,Air India,IndiGo,Jet Airways,Multiple carriers,SpiceJet,Vistara,other_flight
0,1,6,6,0,0,0,1,0,0,17,...,0,0,0,0,0,1,0,0,0,0
1,1,12,5,0,0,1,0,0,0,6,...,0,0,0,0,1,0,0,0,0,0
2,1,21,5,0,0,0,0,1,0,19,...,0,0,0,0,0,1,0,0,0,0
3,1,21,5,0,0,0,0,1,0,8,...,0,0,0,0,0,0,1,0,0,0
4,0,24,6,1,0,0,0,0,0,23,...,1,0,0,0,0,0,0,0,0,0


In [17]:
y_pred_test = voting.predict(X_test)

In [18]:
y_pred_test[:10]

array([14227.40931513,  4696.14263253, 12671.84385   , 11494.93557087,
        3995.74671209, 10104.73628828,  9525.2261275 ,  5485.84340011,
        4781.23484572, 14920.0878266 ])

In [19]:
submission = pd.DataFrame({
    'Price': y_pred_test
})

submission.to_excel("flight_prediction_2.xlsx", index = False)

In [20]:
joblib.dump(voting, 'model_voting.pkl')

['model_voting.pkl']

In [22]:
pickle.dump(voting, open("voting.pickle.dat", "wb"))

In [ ]:
X_test.shape

In [ ]:
X_test.columns

In [ ]:
X = pd.DataFrame({
    'Total_Stops' : [1], 'Date': [1], 'Month': [1], 'M': [1], 'Sa': [1], 'Su': [1], 'Th': [1], 'Tu': [1], 'W': [1],
       'Dep_hrs': [1], 'Dep_min': [1], 'No info': [1], 'other': [1], 'Duration_min': [1],
       'src__Chennai': [1], 'src__Delhi': [1], 'src__Kolkata': [1], 'src__Mumbai': [1],
       'dest__Cochin': [1], 'dest__Delhi': [1], 'dest__Hyderabad': [1], 'dest__Kolkata' : [1],
       'Air India': [1], 'IndiGo': [1], 'Jet Airways': [1], 'Multiple carriers': [1], 'SpiceJet': [1],
       'Vistara': [1], 'other_flight': [1]
})

In [ ]:
voting.predict(X)